In [1]:
import pandas as pd
reviews = pd.read_csv('reviews2.csv')

In [2]:
reviews.shape

(1707861, 9)

In [3]:
comment=reviews.drop(columns=['review_id','user_id','business_id','stars','useful','funny','cool','date'],axis=1)

In [39]:
comment2 = comment.head(100)

In [40]:
import re

comment2['text_processed'] = \
comment2['text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
comment2['text_processed'] = \
comment2['text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
comment2['text_processed'].head()

/Users/zhudi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/zhudi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    very upscale and romantic place good was excep...
1    the food is always great here the service from...
2    literally the slowest and worst service i have...
3    i have been here twice and have had really goo...
4    went here with an friend visiting from italy: ...
Name: text_processed, dtype: object

In [41]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['use','food','good','place'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = comment2.text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['upscale', 'romantic', 'exceptionally', 'appetizer', 'entree', 'finally', 'mouth', 'watering', 'dessert', 'servers', 'pleasant', 'ambiance', 'warm', 'welcoming']


[nltk_data] Downloading package stopwords to /Users/zhudi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
pip install -U gensim

Requirement already up-to-date: gensim in /Users/zhudi/opt/anaconda3/lib/python3.7/site-packages (4.0.1)
Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
os.chdir('..')

In [42]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


In [43]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"back" + 0.007*"time" + 0.006*"really" + 0.006*"well" + 0.005*"first" '
  '+ 0.005*"salad" + 0.005*"great" + 0.005*"got" + 0.005*"super" + '
  '0.005*"like"'),
 (1,
  '0.009*"ordered" + 0.007*"time" + 0.007*"get" + 0.007*"like" + '
  '0.006*"service" + 0.006*"great" + 0.006*"came" + 0.006*"really" + '
  '0.005*"right" + 0.004*"also"'),
 (2,
  '0.008*"like" + 0.008*"great" + 0.006*"little" + 0.005*"well" + '
  '0.005*"really" + 0.005*"made" + 0.005*"ordered" + 0.005*"also" + '
  '0.004*"delicious" + 0.004*"would"'),
 (3,
  '0.012*"great" + 0.009*"best" + 0.009*"like" + 0.008*"service" + 0.007*"one" '
  '+ 0.006*"got" + 0.006*"ever" + 0.006*"get" + 0.005*"also" + 0.005*"really"'),
 (4,
  '0.007*"pizza" + 0.006*"great" + 0.006*"back" + 0.006*"us" + 0.006*"ordered" '
  '+ 0.005*"get" + 0.005*"would" + 0.005*"friend" + 0.005*"also" + '
  '0.004*"service"'),
 (5,
  '0.010*"sushi" + 0.009*"great" + 0.007*"service" + 0.006*"would" + '
  '0.004*"donuts" + 0.004*"friendly" + 0.004*

In [44]:
topics = [lda_model[corpus[i]] for i in range(len(data))]

In [45]:
def topics_document_to_dataframe(topics_document, num_topics):
    res = pd.DataFrame(columns=range(num_topics))
    for topic_weight in topics_document:
        res.loc[0, topic_weight[0]] = topic_weight[1]
    return res

In [46]:
document_topic = \
pd.concat([topics_document_to_dataframe(topics_document, num_topics=num_topics) for topics_document in topics]) \
  .reset_index(drop=True).fillna(0)

In [47]:
document_topic.head()

,0,1,2,3,4,5,6,7,8,9
0,0.939993,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.943738,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.952623,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.993791,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.959085,0.0,0.000000,0.0,0.0,0.0
